# Parallelization Lab

In this lab, you will be leveraging several concepts you have learned to obtain a list of links from a web page and crawl and index the pages referenced by those links - both sequentially and in parallel. Follow the steps below to complete the lab.

## Step 1: Use the requests library to retrieve the content from the URL below.

In [1]:
import requests

url = 'https://en.wikipedia.org/wiki/Data_science'

In [4]:
data_science = requests.get(url).content

## Step 2: Use BeautifulSoup to extract a list of all the unique links on the page.

In [5]:
from bs4 import BeautifulSoup

In [6]:
dt_sc = BeautifulSoup(data_science, 'lxml')

## Step 3: Use list comprehensions with conditions to clean the link list.

There are two types of links, absolute and relative. Absolute links have the full URL and begin with http while relative links begin with a forward slash (/) and point to an internal page within the wikipedia.org domain. Clean the respective types of URLs as follows.

- Absolute Links: Create a list of these and remove any that contain a percentage sign (%).
- Relativel Links: Create a list of these, add the domain to the link so that you have the full URL, and remove any that contain a percentage sign (%).
- Combine the list of absolute and relative links and ensure there are no duplicates.

In [7]:
domain = 'http://wikipedia.org'

In [9]:
raw_links = [str(link) for link in dt_sc.find_all("a", attrs={"href":True})]

In [75]:
absolute_links = [s for l in raw_links for s in l.split() if "http" in s and "%" not in s] 
absolute_links = [s[1:-1] for l in absolute_links for s in l.split("=")]
absolute_links = [s for s in absolute_links if s.startswith("http")]
absolute_links = list(set(absolute_links))

In [76]:
len(absolute_links)

62

In [77]:
relative_links = [s.replace('href="', '') for l in raw_links for s in l.split() if "wiki" in s and "%" not in s]
relative_links = [domain+s for s in relative_links if s.startswith("/wiki")]
relative_links = list(set(relative_links))

In [78]:
len(relative_links)

162

In [82]:
len(set(absolute_links+relative_links)) == len(absolute_links+relative_links)

True

In [127]:
clean_links = absolute_links+relative_links

### No hay duplicados

## Step 4: Use the os library to create a folder called *wikipedia* and make that the current working directory.

In [107]:
import os

In [85]:
os.mkdir('wikipedia/')

In [109]:
#!/usr/bin/python
os.chdir("wikipedia/")

In [113]:
!pwd

/home/miguel/Desktop/Ironhack/datamex0320/module-1/lab-parallelization/your-code/wikipedia


## Step 5: Write a function called index_page that accepts a link and does the following.

- Tries to request the content of the page referenced by that link.
- Slugifies the filename using the `slugify` function from the [python-slugify](https://pypi.org/project/python-slugify/) library and adds a .html file extension.
    - If you don't already have the python-slugify library installed, you can pip install it as follows: `$ pip install python-slugify`.
    - To import the slugify function, you would do the following: `from slugify import slugify`.
    - You can then slugify a link as follows `slugify(link)`.
- Creates a file in the wikipedia folder using the slugified filename and writes the contents of the page to the file.
- If an exception occurs during the process above, just `pass`.

In [116]:
from slugify import slugify

In [147]:
def index_page(link):
    try:
        resp = requests.get(link).content
        soup = BeautifulSoup(resp, 'lxml')
        name = link+'.html'
        name = name.replace('/','_')
        name = name.replace(':', '_')
        with open(name, "w+") as f:
            f.write(str(resp))
        
    except:
            pass

In [149]:
len(clean_links)


224

## Step 6: Sequentially loop through the list of links, running the index_page function each time.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [150]:
%%time
for e in clean_links:
        index_page(e)
        

CPU times: user 41.9 s, sys: 483 ms, total: 42.4 s
Wall time: 4min 55s


## Step 7: Perform the page indexing in parallel and note the difference in performance.

Remember to include `%%time` at the beginning of the cell so that it measures the time it takes for the cell to run.

In [151]:
import multiprocessing

In [152]:
%%time

pool = multiprocessing.Pool()
result = pool.map(index_page, clean_links)
pool.close()


CPU times: user 53.2 ms, sys: 57.1 ms, total: 110 ms
Wall time: 45.9 s
